In [ ]:
import keras
keras.__version__

In [ ]:
#users = ["alex", "ben", "miao", "natasha", "nick", "sarah", "sean", "spencer", "tim", "yijun"]
users = ["alex"]
#mats = ["cloth", "concrete", "door", "drywall", "laminant", "whiteboard"]
mats = ["cloth"]

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import os
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array, load_img

# grab all training file names, count total
files = []
for user in users:
    for mat in mats:
        images = [f for f in os.listdir("data/"+user+"/frames/") if mat in f]
        files.extend(images)
print "Frames found:", len(files)

# create single arrays to store data
X = np.ndarray(shape=(len(files), 224, 224), dtype=np.float32)
y = np.ndarray(shape=(len(files), 1), dtype=np.float32)

# load in all data from images into normalized array
n = 0
for user in users:
    print "  Loading data for", user
    for mat in mats:
        files = [f for f in os.listdir("data/"+user+"/frames/") if mat in f]
        for f in files:
            x = cv2.imread("data/"+user+"/frames/"+f, cv2.IMREAD_GRAYSCALE)
            x = (x - 128.0) / 128.0
            X[n] = x[16:240,16:240]
            y[n] = float("hard" in f)
            n += 1
    print "    Loading", mat, "data complete"
print "All images converted to array!"

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=7)

In [ ]:
np.stack((X_train[i],)*3, axis=-1).reshape(224,224,3,1).shape

## Define CNN Architecture

In [ ]:
from keras import layers, models

# CNN: convolution and pooling
model = models.Sequential()
model.add(layers.Conv2D(8, (3,3), activation='relu', padding='same', input_shape=(256, 256, 1)))
model.add(layers.Conv2D(8, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

# fully connected
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
batch_size = 100
epochs = 50

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1
          )

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

## Use VGG

In [ ]:
from keras.applications import VGG16

vgg = VGG16(weights='imagenet',
           include_top=False,
           input_shape=(224,224,3))

# feed each image through VGG and obtain 7*7*512 features
X_train_feat = np.zeros(shape=(X_train.shape[0],7,7,512))
for i in range(X_train.shape[0]):
    X_train_feat[i] = vgg.predict(np.stack((X_train[i],)*3, axis=-1).reshape(1,224,224,3))
X_train_feat = np.reshape(X_train_feat, (X_train.shape[0], 7*7*512))

In [ ]:
# fully connected
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=7*7*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

batch_size = 100
epochs = 50

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(X_train_feat, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1
          )

In [ ]:
# calculate features for test data
X_test_feat = np.zeros(shape=(X_test.shape[0],7,7,512))
for i in range(X_test.shape[0]):
    X_test_feat[i] = vgg.predict(np.stack((X_test[i],)*3, axis=-1).reshape(1,224,224,3))
X_test_feat = np.reshape(X_test_feat, (X_test.shape[0], 7*7*512))

test_loss, test_acc = model.evaluate(X_test_feat, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)